<a href="https://colab.research.google.com/github/Deepti-Shringare/Downscaling_of_no2map_XGBoost/blob/main/COLAB_CODES/Ground_Satellite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

# Mount Drive
from google.colab import drive
drive.mount('/content/drive')

# --------- PATHS ----------
satellite_csv = "/content/drive/MyDrive/MAJOR_PROJECT/Satellite_dataset/SoniaVihar_SatelliteNO2_2024.csv"
ground_csv = "/content/drive/MyDrive/MAJOR_PROJECT/Dataset_merging_12months/12months_merge_SoniaVihar.csv"

# --------- LOAD DATA ----------
sat_df = pd.read_csv(satellite_csv)
gt_df = pd.read_csv(ground_csv)

# --------- CLEAN ----------
sat_df['date'] = pd.to_datetime(sat_df['date'])
gt_df['date'] = pd.to_datetime(gt_df['date'])

sat_df.rename(columns={'no2': 'tropospheric_no2'}, inplace=True)

# --------- MERGE ----------
merged_df = pd.merge(
    gt_df,
    sat_df[['date', 'tropospheric_no2']],
    on='date',
    how='inner'
)

# Rename columns to match desired output names
merged_df.rename(columns={
    'Station Name': 'station',
    'Latitude': 'lat',
    'Longitude': 'lon',
    'NO2': 'groundtruth_no2' # Assuming 'NO2' from gt_df is groundtruth_no2
}, inplace=True)

# --------- REORDER ----------
merged_df = merged_df[
    ['station', 'lat', 'lon', 'date', 'groundtruth_no2', 'tropospheric_no2']
]

# --------- SAVE ----------
output_path = "/content/drive/MyDrive/MAJOR_PROJECT/Groundtruth_Satellite/GSmerged_SoniaVihar_2024.csv"
merged_df.to_csv(output_path, index=False)

print("✅ Merged dataset saved at:")
print(output_path)

merged_df.head()

var nsit = ee.Geometry.Point([77.0510, 28.7762]);

var collection = ee.ImageCollection('COPERNICUS/S5P/OFFL/L3_NO2')
  .select('tropospheric_NO2_column_number_density')
  .filterBounds(nsit)
  .filterDate('2024-01-01', '2025-01-01');  // ✅ FIXED

// 2024 is a leap year → 366 days
var days = ee.List.sequence(0, 365);

var dailyNO2 = ee.FeatureCollection(days.map(function(d) {
  var start = ee.Date('2024-01-01').advance(d, 'day');
  var end = start.advance(1, 'day');

  var dailyImage = collection.filterDate(start, end).mean();

  var value = dailyImage.reduceRegion({
    reducer: ee.Reducer.mean(),
    geometry: nsit,
    scale: 1000,
    maxPixels: 1e13
  });

  return ee.Feature(null, {
    date: start.format('YYYY-MM-dd'),
    no2: value.get('tropospheric_NO2_column_number_density')
  });
}));

Export.table.toDrive({
  collection: dailyNO2,
  description: 'Bawana',
  fileFormat: 'CSV'
});


***NO2 COLUMN MISSING ISSUES
// 📍 Bawana (buffered)
var site = ee.Geometry.Point([77.0510, 28.7762]).buffer(5000); // 5 km buffer

var collection = ee.ImageCollection('COPERNICUS/S5P/OFFL/L3_NO2')
  .select('tropospheric_NO2_column_number_density')
  .filterBounds(site)
  .filterDate('2024-01-01', '2025-01-01');

// 2024 = leap year
var days = ee.List.sequence(0, 365);

var dailyNO2 = ee.FeatureCollection(days.map(function(d) {
  var start = ee.Date('2024-01-01').advance(d, 'day');
  var end = start.advance(1, 'day');

  var img = collection.filterDate(start, end)
    .mean()
    .unmask(-9999);   // 🔥 FORCE VALUE

  var value = img.reduceRegion({
    reducer: ee.Reducer.mean(),
    geometry: site,
    scale: 1000,
    maxPixels: 1e13
  });

  return ee.Feature(null, {
    date: start.format('YYYY-MM-dd'),
    no2: value.get('tropospheric_NO2_column_number_density')
  });
}));

Export.table.toDrive({
  collection: dailyNO2,
  description: 'Bawana_NO2_2024_FIXED',
  fileFormat: 'CSV'
});
